Function call，简单的理解就是由LLM分析用户的输入，如果需要调用事先定义好的function，LLM返回function名字以及要传递给function的参数。用户空间执行function，再把function执行结果返回给LLM继续分析，如此循环，直到LLM认为得到最终答案，LLM返回最终答案。  
  
每一种模型对function call的格式有自己的定义，这和训练模型时使用的数据有关，所以对于不同模型，function call的处理可能不同。  
  
参考 https://qwen.readthedocs.io/en/v2.0/framework/function_call.html

In [1]:
import json
import re

# function call实现函数，用于取得当前温度
def get_current_temperature(location: str, unit: str = "celsius"):
    """Get current temperature at a location.

    Args:
        location: The location to get the temperature for, in the format "City, State, Country".
        unit: The unit to return the temperature in. Defaults to "celsius". (choices: ["celsius", "fahrenheit"])

    Returns:
        the temperature, the location, and the unit in a dict
    """
    return {
        "temperature": 26.1,
        "location": location,
        "unit": unit,
    }

# function call实现函数，用于取得date指定的日期的温度
def get_temperature_date(location: str, date: str, unit: str = "celsius"):
    """Get temperature at a location and date.

    Args:
        location: The location to get the temperature for, in the format "City, State, Country".
        date: The date to get the temperature for, in the format "Year-Month-Day".
        unit: The unit to return the temperature in. Defaults to "celsius". (choices: ["celsius", "fahrenheit"])

    Returns:
        the temperature, the location, the date and the unit in a dict
    """
    return {
        "temperature": 25.9,
        "location": location,
        "date": date,
        "unit": unit,
    }

# 根据name返回对应的函数，用法如下：
# fn_res: str = json.dumps(get_function_by_name(fn_name)(**fn_args))
def get_function_by_name(name):
    if name == "get_current_temperature":
        return get_current_temperature
    if name == "get_temperature_date":
        return get_temperature_date

def try_parse_tool_calls(content: str):
    """Try parse the tool calls."""
    tool_calls = []
    for m in re.finditer(r"<\|tool_call_start\|>(.+)?<\|tool_call_end\|>", content):
        try:
            func = json.loads(m.group(1))
            tool_calls.append({"type": "function", "function": func})
            if isinstance(func["arguments"], str):
                func["arguments"] = json.loads(func["arguments"])
        except json.JSONDecodeError as _:
            print(m)
            pass
    if tool_calls:
        return {"role": "assistant", "tool_calls": tool_calls}
    return {"role": "assistant", "content": re.sub(r"<\|im_end\|>$", "", content)}

# TOOLS是一个列表，这里定义了2个元素，每个元素都是一个function，其用法如下：
# functions = [tool["function"] for tool in TOOLS]
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_current_temperature",
            "description": "Get current temperature at a location.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": 'The location to get the temperature for, in the format "City, State, Country".',
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": 'The unit to return the temperature in. Defaults to "celsius".',
                    },
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_temperature_date",
            "description": "Get temperature at a location and date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": 'The location to get the temperature for, in the format "City, State, Country".',
                    },
                    "date": {
                        "type": "string",
                        "description": 'The date to get the temperature for, in the format "Year-Month-Day".',
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": 'The unit to return the temperature in. Defaults to "celsius".',
                    },
                },
                "required": ["location", "date"],
            },
        },
    },
]

# 消息列表，初始时包含2条消息，后面与LLM的交互的消息都会追加保存到这个消息列表中
MESSAGES = [
    {"role": "system", "content": "You are a helpful assistant.\n\nCurrent Date: 2024-08-31"},
    {"role": "user",  "content": "What's the temperature in San Francisco now? How about tomorrow?"},
]

In [2]:
from qwen_agent.llm import get_chat_model

# 基于qwen_agent库取得模型，使用ollama本地部署的qwen2
llm = get_chat_model({
    "model": "qwen2",
    "model_server": "http://127.0.0.1:11434/v1",
    "api_key": "EMPTY",
})

In [3]:
# 初始化消息列表和functions列表
messages = MESSAGES[:]
functions = [tool["function"] for tool in TOOLS]

In [4]:
# 与模型交互，传递消息列表和functions列表
for responses in llm.chat(
    messages=messages,
    functions=functions,
    extra_generate_cfg=dict(parallel_function_calls=True),    # parallel_function_calls设置为True，表示可以并行处理多个function
):
    pass
messages.extend(responses)    # LLM的返回信息追加到消息列表中

In [5]:
#查看第一次与LLM交互后的消息列表可以发现，LLM同时返回2条消息，‘role’是assistant，'content'都是空，'function_call'指定了要调用的function的名字，'arguments'指定了要传递给function的参数。
messages

[{'role': 'system',
  'content': 'You are a helpful assistant.\n\nCurrent Date: 2024-08-31'},
 {'role': 'user',
  'content': "What's the temperature in San Francisco now? How about tomorrow?"},
 {'role': 'assistant',
  'content': '',
  'function_call': {'name': 'get_current_temperature',
   'arguments': '{"location": "San Francisco, CA, USA", "unit": "celsius"}'}},
 {'role': 'assistant',
  'content': '',
  'function_call': {'name': 'get_temperature_date',
   'arguments': '{"location": "San Francisco, CA, USA", "date": "2024-08-31", "unit": "fahrenheit"}'}}]

In [6]:
# 查看第一次与LLM交互后，LLM返回的消息
responses

[{'role': 'assistant',
  'content': '',
  'function_call': {'name': 'get_current_temperature',
   'arguments': '{"location": "San Francisco, CA, USA", "unit": "celsius"}'}},
 {'role': 'assistant',
  'content': '',
  'function_call': {'name': 'get_temperature_date',
   'arguments': '{"location": "San Francisco, CA, USA", "date": "2024-08-31", "unit": "fahrenheit"}'}}]

In [7]:
# 执行LLM要求执行的function，并将function执行结果追加到消息列表中

for message in responses:
    if fn_call := message.get("function_call", None):
        fn_name: str = fn_call['name']
        fn_args: dict = json.loads(fn_call["arguments"])

        fn_res: str = json.dumps(get_function_by_name(fn_name)(**fn_args))

        messages.append({
            "role": "function",
            "name": fn_name,
            "content": fn_res,
        })

In [8]:
# 查看消息列表，function执行结果的'role'是function，'name'是function名字，'content'是function执行返回的结果。这是上一步追加消息时刻意设定的格式。
messages

[{'role': 'system',
  'content': 'You are a helpful assistant.\n\nCurrent Date: 2024-08-31'},
 {'role': 'user',
  'content': "What's the temperature in San Francisco now? How about tomorrow?"},
 {'role': 'assistant',
  'content': '',
  'function_call': {'name': 'get_current_temperature',
   'arguments': '{"location": "San Francisco, CA, USA", "unit": "celsius"}'}},
 {'role': 'assistant',
  'content': '',
  'function_call': {'name': 'get_temperature_date',
   'arguments': '{"location": "San Francisco, CA, USA", "date": "2024-08-31", "unit": "fahrenheit"}'}},
 {'role': 'function',
  'name': 'get_current_temperature',
  'content': '{"temperature": 26.1, "location": "San Francisco, CA, USA", "unit": "celsius"}'},
 {'role': 'function',
  'name': 'get_temperature_date',
  'content': '{"temperature": 25.9, "location": "San Francisco, CA, USA", "date": "2024-08-31", "unit": "fahrenheit"}'}]

In [9]:
# 得到function执行结果后，再再次与LLM交互。消息列表中包含了function执行结果。
for responses in llm.chat(messages=messages, functions=functions):
    pass
messages.extend(responses)

In [10]:
# 查看LLM的响应信息
responses

[{'role': 'assistant',
  'content': 'The current temperature in San Francisco is approximately 26.1 degrees Celsius. For tomorrow, the forecast predicts a temperature of about 25.9 degrees Fahrenheit in San Francisco.'}]

In [11]:
# 查看消息列表
messages

[{'role': 'system',
  'content': 'You are a helpful assistant.\n\nCurrent Date: 2024-08-31'},
 {'role': 'user',
  'content': "What's the temperature in San Francisco now? How about tomorrow?"},
 {'role': 'assistant',
  'content': '',
  'function_call': {'name': 'get_current_temperature',
   'arguments': '{"location": "San Francisco, CA, USA", "unit": "celsius"}'}},
 {'role': 'assistant',
  'content': '',
  'function_call': {'name': 'get_temperature_date',
   'arguments': '{"location": "San Francisco, CA, USA", "date": "2024-08-31", "unit": "fahrenheit"}'}},
 {'role': 'function',
  'name': 'get_current_temperature',
  'content': '{"temperature": 26.1, "location": "San Francisco, CA, USA", "unit": "celsius"}'},
 {'role': 'function',
  'name': 'get_temperature_date',
  'content': '{"temperature": 25.9, "location": "San Francisco, CA, USA", "date": "2024-08-31", "unit": "fahrenheit"}'},
 {'role': 'assistant',
  'content': 'The current temperature in San Francisco is approximately 26.1 degr